# Preprocessing 🗂

> Preprocessing of csv data.

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | default_exp preprocessing

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import enum
import glob
from typing import Optional, Literal, get_args
from dataclasses import dataclass
import re
import pandas as pd
import numpy as np
import os
import datetime
from pprint import pprint
from functools import partial

In [ ]:
# | hide
# | export
@enum.unique
class Event(enum.Enum):
    "A class to represent the event type of a LOBSTER message."
    UNKNOWN = 0
    SUBMISSION = 1
    CANCELLATION = 2
    DELETION = 3
    EXECUTION = 4
    HIDDEN_EXECUTION = 5
    CROSS_TRADE = 6
    OTHER = 8
    TRADING_HALT = 9
    RESUME_QUOTE = 10
    TRADING_RESUME = 11


@enum.unique
class EventGroup(enum.Enum):
    EXECUTIONS = [Event.EXECUTION.value, Event.HIDDEN_EXECUTION.value]
    HALTS = [
        Event.TRADING_HALT.value,
        Event.RESUME_QUOTE.value,
        Event.TRADING_RESUME.value,
    ]
    CANCELLATIONS = [Event.CANCELLATION.value, Event.DELETION.value]

The event column in the lobster data encodes the following.

In [ ]:
# | echo: false
{e.name: e.value for e in Event}

{'UNKNOWN': 0,
 'SUBMISSION': 1,
 'CANCELLATION': 2,
 'DELETION': 3,
 'EXECUTION': 4,
 'HIDDEN_EXECUTION': 5,
 'CROSS_TRADE': 6,
 'OTHER': 8,
 'TRADING_HALT': 9,
 'RESUME_QUOTE': 10,
 'TRADING_RESUME': 11}

::: {.callout-note}
Here, trading halts are split into three categories: trading halt, resume quote and trading resume. To access all three types of trading halt events, use `EventGroup.TRADING_HALTS`.
:::

To access common groups of events, an `EventGroup` is defined.

In [ ]:
# | echo: false
pprint({e.name: e.value for e in EventGroup}, width=25)

{'CANCELLATIONS': [2, 3],
 'EXECUTIONS': [4, 5],
 'HALTS': [9, 10, 11]}


In [ ]:
# | hide
# | export


@enum.unique
class Direction(enum.Enum):
    BUY = 1
    SELL = -1

The csv data is assumed to be stored roughly as follows. There should be one folder per ticker, with underscores separating the ticker name, start date, end date and number of levels. Each individual csv filename should also end with the number of levels. All dates should be listed as %YYYY-%MM-%DD convention.

```bash
csv_lobster_data
├── AAPL_2016-06-21_2016-06-21_10
│   ├── AAPL_2012-06-21_34200000_57600000_message_10.csv
│   ├── AAPL_2012-06-21_34200000_57600000_orderbook_10.csv
│   ├── AAPL_2012-06-22_34200000_57600000_message_10.csv
│   └── AAPL_2012-06-22_34200000_57600000_orderbook_10.csv
├── GOOG_2016-06-21_2016-06-22_10
│   ├── GOOG_2012-06-21_34200000_57600000_message_10.csv
│   ├── GOOG_2012-06-21_34200000_57600000_orderbook_10.csv
│   ├── GOOG_2012-06-22_34200000_57600000_message_10.csv
│   └── GOOG_2012-06-22_34200000_57600000_orderbook_10.csv
└── LOBSTER_SampleFiles_ReadMe.txt
```

The `Data` object stores information about the data to be loaded, as well as specifying which preprocessing options are to be applied.

In [ ]:
# | export

LoadType = Literal["both", "messages", "book"]
TickerTypes = Literal[None, "equity", "etf"]

(None, 'equity', 'etf')

In [ ]:
# | exports
# | code-fold: true


@dataclass
class Data:
    directory_path: str = "/home/petit/Documents/data/lobster/csv"  # path to data
    ticker: str = "AAPL"
    date_range: Optional[str | tuple[str, str]] = None
    levels: Optional[int] = None
    nrows: Optional[int] = None
    load: LoadType = "messages"
    add_ticker: bool = True
    ticker_type: TickerTypes = None
    clip_trading_hours: bool = True
    aggregate_duplicates: bool = True

    def __post_init__(self) -> None:
        assert self.load in get_args(LoadType)
        assert self.ticker_type in get_args(TickerTypes)

        # ticker path
        tickers = glob.glob(f"{self.directory_path}/*")
        ticker_path = [t for t in tickers if self.ticker in t]
        assert len(ticker_path) == 1
        self.ticker_path = ticker_path[0]

        # levels
        if not self.levels:
            self.levels = int(self.ticker_path.split("_")[-1])
            assert self.levels >= 1

        # infer date range from ticker folder name
        if not self.date_range:
            self.date_range = tuple(re.findall(pattern=r"\d\d\d\d-\d\d-\d\d", string=self.ticker_path))
            assert len(self.date_range) == 2

        # book and message paths
        tickers = glob.glob(f"{self.ticker_path}/*")
        tickers_end = list(map(os.path.basename, tickers))

        if isinstance(self.date_range, tuple):
            # get all dates in folder
            dates = set([re.findall(pattern=r"\d\d\d\d-\d\d-\d\d", string=file)[0] for file in tickers_end])
            # filter for dates within specified range
            dates = sorted(
                list(
                    filter(
                        lambda date: self.date_range[0] <= date <= self.date_range[1],
                        dates,
                    )
                )
            )

            self.dates = dates
            self.date_range = (min(self.dates), max(self.dates))

        elif isinstance(self.date_range, str):
            self.dates, self.date_range = [self.date_range], (
                self.date_range,
                self.date_range,
            )

        # messages and book filepath dictionaries
        def _create_date_to_path_dict(keyword: str) -> dict:
            filter_keyword_tickers = list(filter(lambda x: keyword in x, tickers_end))
            date_path_dict = {}
            for date in self.dates:
                filter_date_tickers = list(filter(lambda x: date in x, filter_keyword_tickers))
                assert len(filter_date_tickers) == 1
                date_path_dict[date] = os.path.join(self.ticker_path, filter_date_tickers[0])
            return date_path_dict

        self.book_paths = _create_date_to_path_dict("book")
        self.messages_paths = _create_date_to_path_dict("message")

In [ ]:
# | hide
# | export
def aggregate_duplicates_(df: pd.DataFrame) -> pd.DataFrame:
    subset = list(df.columns)
    subset.remove("size")
    subset.insert(0, "datetime")

    duplicates_index = df.reset_index().duplicated(subset=subset, keep=False).values
    duplicates = df[duplicates_index]
    no_duplicates = df[~duplicates_index]

    assert no_duplicates.shape[0] + duplicates.shape[0] == df.shape[0]

    aggregated_duplicates = duplicates.groupby(by=[duplicates.index, "direction"]).agg({"size": "sum"})
    aggregated_duplicates = pd.merge(duplicates, aggregated_duplicates, on="datetime", suffixes=("_original", "_agg"))
    aggregated_duplicates.drop(columns=["size_original"], inplace=True)
    aggregated_duplicates.rename(columns={"size_agg": "size"}, inplace=True)
    aggregated_duplicates.drop_duplicates(keep="first", inplace=True)

    df = pd.concat([no_duplicates, aggregated_duplicates])
    df.sort_index()
    return df

The `Lobster` loads the csv data into its `messages` and `book` attributes. The data to be loaded is passed in as a `Data` object.

In [ ]:
# | exports
# | code-fold: true


@dataclass
class Lobster:
    "Lobster data class for a single symbol of Lobster data."
    data: Data = Data()

    def __post_init__(self):
        if self.data.load in ["messages", "both"]:
            dfs = []
            for date, filepath in self.data.messages_paths.items():
                # load messages
                df = pd.read_csv(
                    filepath,
                    header=None,
                    nrows=self.data.nrows,
                    usecols=list(range(6)),
                    names=["time", "event", "order_id", "size", "price", "direction"],
                    index_col=False,
                    dtype={
                        "time": "float64",
                        "event": "uint8",
                        "price": "int64",
                        "direction": "int8",
                        "order_id": "uint32",
                        "size": "uint64",
                    },
                )

                # set index as datetime
                df["datetime"] = pd.to_datetime(date, format="%Y-%m-%d") + df.time.apply(lambda x: pd.to_timedelta(x, unit="s"))
                df.set_index("datetime", drop=True, inplace=True)
                df.drop(columns="time", inplace=True)
                dfs.append(df)
            df = pd.concat(dfs)

            # use 0 as NaN for price, size and direction
            assert df.loc[df.event.eq(Event.TRADING_HALT.value), "direction"].eq(-1).all()
            df.loc[df.event.eq(Event.TRADING_HALT.value), "direction"] = 0

            # process trading halts
            def _trading_halt_type(price):
                return {
                    -1: Event.TRADING_HALT.value,
                    0: Event.RESUME_QUOTE.value,
                    1: Event.TRADING_RESUME.value,
                }[price]

            df.loc[df.event.eq(Event.TRADING_HALT.value), "event"] = df.loc[df.event.eq(Event.TRADING_HALT.value), "price"].apply(
                lambda x: _trading_halt_type(x)
            )

            df.loc[
                df.event.isin(EventGroup.HALTS.value),
                ["order_id", "size", "price"],
            ] = [
                0,
                0,
                np.nan,
            ]

            # set price in dollars
            df.price = df.price.apply(lambda x: x / 10_000).astype("float64")

            if self.data.add_ticker:
                df = df.assign(ticker=self.data.ticker).astype({"ticker": "category"})

            if self.data.ticker_type:
                assert self.data.ticker_type in [
                    "equity",
                    "etf",
                ], "ticker_type must be either `equity` or `etf`"
                df = df.assign(ticker_type=self.data.ticker_type).astype(
                    dtype={"ticker_type": pd.CategoricalDtype(categories=["equity", "etf"])}
                )

            self.messages = df

        if self.data.load in ["book", "both"]:
            col_names = []
            for level in range(1, self.data.levels + 1):
                for col_type in ["ask_price", "ask_size", "bid_price", "bid_size"]:
                    col_name = f"{col_type}_{level}"
                    col_names.append(col_name)

            # for now just use float64
            # col_dtypes = {
            #     col_name: pd.Int64Dtype() if ("size" in col_name) else "float"
            #     for col_name in col_names
            # }

            dfs = []
            for filename in self.data.book_paths.values():
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=self.data.nrows,
                    usecols=list(range(4 * self.data.levels)),
                    names=col_names,
                    dtype="float64",
                    na_values=[-9999999999, 9999999999, 0],
                )

                dfs.append(df)
            df = pd.concat(dfs)
            
            df.set_index(self.messages.index, inplace=True, drop=True)

            price_cols = df.columns.str.contains("price")
            df.loc[:, price_cols] = df.loc[:, price_cols].apply(lambda x: x / 10_000)

            self.book = df

        # data cleaning on messages done now, as book infers times from messages file
        # aggregate duplicates 
        if self.data.aggregate_duplicates:
            self.messages = aggregate_duplicates_(self.messages)

        # clip messages to trading hours (from 9:30 to 4:30)
        if self.data.clip_trading_hours:
            df = df.iloc[(df.index.time >= datetime.time(9, 30)) & (df.index.time < datetime.time(16, 30))]

            
    def __repr__(self) -> str:
        return f"Lobster data for ticker: {self.data.ticker} for date range: {self.data.date_range[0]} to {self.data.date_range[1]}."

The `load_lobster` is a simple function which returns a `Lobster` object.

In [ ]:
# | export


def load_lobster(**kwargs):
    """Load `Lobster` object from csv data."""
    data = Data(**kwargs)
    lobster = Lobster(data)

    return lobster

To overwrite defaults `functools.partial` can be applied.

In [ ]:
# | echo: true
my_load_lobster = partial(load_lobster, directory_path="/my/path")


In [ ]:
# | hide
load_lobster = partial(load_lobster, directory_path="/home/petit/Documents/data/csv_lobster_data")

In [ ]:
lobster = load_lobster(ticker="AAPL", date_range="2012-06-21", ticker_type="equity")
lobster.messages.head()


,event,order_id,size,price,direction,ticker,ticker_type
datetime,,,,,,,
2012-06-21 09:30:00.004241176,1,16113575,18,585.33,1,AAPL,equity
2012-06-21 09:30:00.004260640,1,16113584,18,585.32,1,AAPL,equity
2012-06-21 09:30:00.004447484,1,16113594,18,585.31,1,AAPL,equity
2012-06-21 09:30:00.025551909,1,16120456,18,585.91,-1,AAPL,equity
2012-06-21 09:30:00.025579546,1,16120480,18,585.92,-1,AAPL,equity


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()